<a href="https://colab.research.google.com/github/carlosprr29/ai-progetto-spagnoli/blob/main/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup Environment

In [1]:
##Celda de Instalación y carga
!pip install -q datasets pandas scikit-learn

from datasets import load_dataset
import pandas as pd

# Descargamos el dataset WELFake
print("⏳ Descargando dataset...")
dataset_raw = load_dataset("davanstrien/WELFake")
df = pd.DataFrame(dataset_raw['train'])

# Limpieza rápida: quitamos filas que no tengan texto o título
df = df.dropna(subset=['title', 'text'])
print(f"✅ Dataset cargado: {len(df)} filas.")


⏳ Descargando dataset...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-290868f0a36350(…):   0%|          | 0.00/152M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/72134 [00:00<?, ? examples/s]

✅ Dataset cargado: 71537 filas.


In [2]:
#Celda de División de Datos
from sklearn.model_selection import train_test_split

# Dividimos el dataset (80% entrenamiento, 20% prueba)
# 'stratify' asegura que haya la misma proporción de noticias reales y fakes en ambos grupos
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

print(f"Datos para entrenar: {len(train_df)}")
print(f"Datos para evaluar: {len(test_df)}")



Datos para entrenar: 57229
Datos para evaluar: 14308


In [3]:
## Experimento 1: Baseline con Regresión Logística
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Definimos la función de entrenamiento y evaluación
def entrenar_y_evaluar(X_train, X_test, y_train, y_test, nombre_experimento):
    # 1. Creamos el modelo (TF-IDF + Regresión Logística)
    # TF-IDF convierte palabras en números según su importancia (max_features evita que sea muy pesado)
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),
        ('clf', LogisticRegression(max_iter=1000))
    ])

    # 2. Entrenamos
    print(f"\n🚀 Entrenando modelo para: {nombre_experimento}...")
    pipeline.fit(X_train, y_train)

    # 3. Predecimos
    predicciones = pipeline.predict(X_test)

    # 4. Resultados
    acc = accuracy_score(y_test, predicciones)
    print(f"✅ Precisión ({nombre_experimento}): {acc:.4f}")
    print(classification_report(y_test, predicciones))

    return acc

# --- EJECUCIÓN DEL ESTUDIO DE ABLACIÓN ---

# CASO 1: Solo con el título
acc_titulos = entrenar_y_evaluar(
    train_df['title'], test_df['title'],
    train_df['label'], test_df['label'],
    "SOLO TÍTULOS"
)

# CASO 2: Título + Texto (Creamos la columna 'total' justo antes)
train_df['total'] = train_df['title'] + " " + train_df['text']
test_df['total'] = test_df['title'] + " " + test_df['text']

acc_completo = entrenar_y_evaluar(
    train_df['total'], test_df['total'],
    train_df['label'], test_df['label'],
    "TEXTO COMPLETO"
)


🚀 Entrenando modelo para: SOLO TÍTULOS...
✅ Precisión (SOLO TÍTULOS): 0.8937
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      7006
           1       0.89      0.90      0.90      7302

    accuracy                           0.89     14308
   macro avg       0.89      0.89      0.89     14308
weighted avg       0.89      0.89      0.89     14308


🚀 Entrenando modelo para: TEXTO COMPLETO...
✅ Precisión (TEXTO COMPLETO): 0.9436
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      7006
           1       0.94      0.95      0.95      7302

    accuracy                           0.94     14308
   macro avg       0.94      0.94      0.94     14308
weighted avg       0.94      0.94      0.94     14308



# **Eficacia del Baseline:** Un 94.36% en texto completo es una marca altísima. Esto indica que el dataset WELFake tiene patrones de palabras muy claros que permiten distinguir noticias reales de fakes con facilidad.

#**Ablation Study (Hallazgo inicial)**: Hay una diferencia de 5 puntos (89% vs 94%).

#**Conclusión:** Aunque el título por sí solo es muy informativo, el cuerpo de la noticia aporta un contexto necesario para captar un 5% extra de casos que el titular no revela.

#**El reto para BERT:** Vuestro Baseline es tan bueno que el desafío ahora no es solo "superar" el 94%, sino ver si BERT es capaz de entender noticias más ambiguas donde las palabras clave no son tan obvias.
